In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col, when, avg, rank
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("EmployeeWorkData").getOrCreate()


In [2]:
data = [
    Row(EmpID=101, Name="Ravi", Department="Engineering", Project="AI Engine", Salary=95000, HoursPerWeek=42),
    Row(EmpID=102, Name="Sneha", Department="Engineering", Project="Data Platform", Salary=87000, HoursPerWeek=45),
    Row(EmpID=103, Name="Kabir", Department="Marketing", Project="Product Launch", Salary=65000, HoursPerWeek=40),
    Row(EmpID=104, Name="Anita", Department="Sales", Project="Client Outreach", Salary=70000, HoursPerWeek=38),
    Row(EmpID=105, Name="Divya", Department="Engineering", Project="AI Engine", Salary=99000, HoursPerWeek=48),
    Row(EmpID=106, Name="Amit", Department="Marketing", Project="Social Media", Salary=62000, HoursPerWeek=35),
    Row(EmpID=107, Name="Priya", Department="HR", Project="Policy Revamp", Salary=58000, HoursPerWeek=37),
    Row(EmpID=108, Name="Manav", Department="Sales", Project="Lead Gen", Salary=73000, HoursPerWeek=41),
    Row(EmpID=109, Name="Neha", Department="Engineering", Project="Security Suite", Salary=91000, HoursPerWeek=46),
    Row(EmpID=110, Name="Farah", Department="HR", Project="Onboarding", Salary=60000, HoursPerWeek=36)
]

df = spark.createDataFrame(data)
df.show(truncate=False)


+-----+-----+-----------+---------------+------+------------+
|EmpID|Name |Department |Project        |Salary|HoursPerWeek|
+-----+-----+-----------+---------------+------+------------+
|101  |Ravi |Engineering|AI Engine      |95000 |42          |
|102  |Sneha|Engineering|Data Platform  |87000 |45          |
|103  |Kabir|Marketing  |Product Launch |65000 |40          |
|104  |Anita|Sales      |Client Outreach|70000 |38          |
|105  |Divya|Engineering|AI Engine      |99000 |48          |
|106  |Amit |Marketing  |Social Media   |62000 |35          |
|107  |Priya|HR         |Policy Revamp  |58000 |37          |
|108  |Manav|Sales      |Lead Gen       |73000 |41          |
|109  |Neha |Engineering|Security Suite |91000 |46          |
|110  |Farah|HR         |Onboarding     |60000 |36          |
+-----+-----+-----------+---------------+------+------------+



In [7]:
# Local view
df.createOrReplaceTempView("employees_local")

# Global view
df.createOrReplaceGlobalTempView("employees_global")



In [9]:
#PART-A
# 1. Get all employees working on the "AI Engine" project
spark.sql("SELECT * FROM employees_local WHERE Project = 'AI Engine'").show()



+-----+-----+-----------+---------+------+------------+
|EmpID| Name| Department|  Project|Salary|HoursPerWeek|
+-----+-----+-----------+---------+------+------------+
|  101| Ravi|Engineering|AI Engine| 95000|          42|
|  105|Divya|Engineering|AI Engine| 99000|          48|
+-----+-----+-----------+---------+------+------------+



In [10]:
# 2. Get Marketing employees with salary greater than 60000
spark.sql("SELECT * FROM employees_local WHERE Department = 'Marketing' AND Salary > 60000").show()


+-----+-----+----------+--------------+------+------------+
|EmpID| Name|Department|       Project|Salary|HoursPerWeek|
+-----+-----+----------+--------------+------+------------+
|  103|Kabir| Marketing|Product Launch| 65000|          40|
|  106| Amit| Marketing|  Social Media| 62000|          35|
+-----+-----+----------+--------------+------+------------+



In [11]:
# 3. Show average salary for each department
spark.sql("SELECT Department, AVG(Salary) AS AvgSalary FROM employees_local GROUP BY Department").show()


+-----------+---------+
| Department|AvgSalary|
+-----------+---------+
|      Sales|  71500.0|
|Engineering|  93000.0|
|  Marketing|  63500.0|
|         HR|  59000.0|
+-----------+---------+



In [12]:
# 4. Display top 3 highest paid employees
spark.sql("SELECT * FROM employees_local ORDER BY Salary DESC LIMIT 3").show()


+-----+-----+-----------+--------------+------+------------+
|EmpID| Name| Department|       Project|Salary|HoursPerWeek|
+-----+-----+-----------+--------------+------+------------+
|  105|Divya|Engineering|     AI Engine| 99000|          48|
|  101| Ravi|Engineering|     AI Engine| 95000|          42|
|  109| Neha|Engineering|Security Suite| 91000|          46|
+-----+-----+-----------+--------------+------+------------+



In [13]:
# 5. Find employees who work more than 40 hours per week
spark.sql("SELECT * FROM employees_local WHERE HoursPerWeek > 40").show()


+-----+-----+-----------+--------------+------+------------+
|EmpID| Name| Department|       Project|Salary|HoursPerWeek|
+-----+-----+-----------+--------------+------+------------+
|  101| Ravi|Engineering|     AI Engine| 95000|          42|
|  102|Sneha|Engineering| Data Platform| 87000|          45|
|  105|Divya|Engineering|     AI Engine| 99000|          48|
|  108|Manav|      Sales|      Lead Gen| 73000|          41|
|  109| Neha|Engineering|Security Suite| 91000|          46|
+-----+-----+-----------+--------------+------+------------+



In [14]:
# 6. Count number of employees in each project
spark.sql("SELECT Project, COUNT(*) AS EmployeeCount FROM employees_local GROUP BY Project").show()


+---------------+-------------+
|        Project|EmployeeCount|
+---------------+-------------+
|  Data Platform|            1|
|      AI Engine|            2|
| Product Launch|            1|
|Client Outreach|            1|
| Security Suite|            1|
|  Policy Revamp|            1|
|       Lead Gen|            1|
|   Social Media|            1|
|     Onboarding|            1|
+---------------+-------------+



In [15]:
# 7. Drop the local view (this removes it from current Spark session)
spark.catalog.dropTempView("employees_local")


True

In [16]:
#PART-B
df.createOrReplaceGlobalTempView("employees_global")


In [17]:
# 1. Get all HR employees working less than 38 hours per week
spark.sql("SELECT * FROM global_temp.employees_global WHERE Department = 'HR' AND HoursPerWeek < 38").show()


+-----+-----+----------+-------------+------+------------+
|EmpID| Name|Department|      Project|Salary|HoursPerWeek|
+-----+-----+----------+-------------+------+------------+
|  107|Priya|        HR|Policy Revamp| 58000|          37|
|  110|Farah|        HR|   Onboarding| 60000|          36|
+-----+-----+----------+-------------+------+------------+



In [18]:
# 2. Show total salary payout for each department
spark.sql("SELECT Department, SUM(Salary) AS TotalPayout FROM global_temp.employees_global GROUP BY Department").show()


+-----------+-----------+
| Department|TotalPayout|
+-----------+-----------+
|      Sales|     143000|
|Engineering|     372000|
|  Marketing|     127000|
|         HR|     118000|
+-----------+-----------+



In [19]:
# 3. Add Status column: 'Overworked' if > 45 hours, else 'Normal'
df_status = df.withColumn("Status", when(col("HoursPerWeek") > 45, "Overworked").otherwise("Normal"))
df_status.createOrReplaceGlobalTempView("employees_status")
spark.sql("SELECT * FROM global_temp.employees_status").show()


+-----+-----+-----------+---------------+------+------------+----------+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|    Status|
+-----+-----+-----------+---------------+------+------------+----------+
|  101| Ravi|Engineering|      AI Engine| 95000|          42|    Normal|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|    Normal|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|    Normal|
|  104|Anita|      Sales|Client Outreach| 70000|          38|    Normal|
|  105|Divya|Engineering|      AI Engine| 99000|          48|Overworked|
|  106| Amit|  Marketing|   Social Media| 62000|          35|    Normal|
|  107|Priya|         HR|  Policy Revamp| 58000|          37|    Normal|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|    Normal|
|  109| Neha|Engineering| Security Suite| 91000|          46|Overworked|
|  110|Farah|         HR|     Onboarding| 60000|          36|    Normal|
+-----+-----+-----------+---------------+------+---

In [20]:
# 4. Count number of employees in each project
spark.sql("SELECT Project, COUNT(*) AS EmployeeCount FROM global_temp.employees_global GROUP BY Project").show()


+---------------+-------------+
|        Project|EmployeeCount|
+---------------+-------------+
|  Data Platform|            1|
|      AI Engine|            2|
| Product Launch|            1|
|Client Outreach|            1|
| Security Suite|            1|
|  Policy Revamp|            1|
|       Lead Gen|            1|
|   Social Media|            1|
|     Onboarding|            1|
+---------------+-------------+



In [21]:
# 5. Show employees whose salary is above department average
spark.sql("""
  WITH dept_avg AS (
    SELECT Department, AVG(Salary) AS avg_salary
    FROM global_temp.employees_global
    GROUP BY Department
  )
  SELECT e.*
  FROM global_temp.employees_global e
  JOIN dept_avg d ON e.Department = d.Department
  WHERE e.Salary > d.avg_salary
""").show()


+-----+-----+-----------+--------------+------+------------+
|EmpID| Name| Department|       Project|Salary|HoursPerWeek|
+-----+-----+-----------+--------------+------+------------+
|  101| Ravi|Engineering|     AI Engine| 95000|          42|
|  105|Divya|Engineering|     AI Engine| 99000|          48|
|  103|Kabir|  Marketing|Product Launch| 65000|          40|
|  108|Manav|      Sales|      Lead Gen| 73000|          41|
|  110|Farah|         HR|    Onboarding| 60000|          36|
+-----+-----+-----------+--------------+------+------------+



In [22]:
# 6. Query global_temp.employees_global in the current session
spark.sql("SELECT * FROM global_temp.employees_global").show()


+-----+-----+-----------+---------------+------+------------+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|
+-----+-----+-----------+---------------+------+------------+
|  101| Ravi|Engineering|      AI Engine| 95000|          42|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|
|  104|Anita|      Sales|Client Outreach| 70000|          38|
|  105|Divya|Engineering|      AI Engine| 99000|          48|
|  106| Amit|  Marketing|   Social Media| 62000|          35|
|  107|Priya|         HR|  Policy Revamp| 58000|          37|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|
|  109| Neha|Engineering| Security Suite| 91000|          46|
|  110|Farah|         HR|     Onboarding| 60000|          36|
+-----+-----+-----------+---------------+------+------------+



In [23]:
#BONUS
# 1. Use window function to rank employees by salary within each department
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

windowSpec = Window.partitionBy("Department").orderBy(col("Salary").desc())
df_ranked = df.withColumn("Rank", rank().over(windowSpec))
df_ranked.show()


+-----+-----+-----------+---------------+------+------------+----+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|Rank|
+-----+-----+-----------+---------------+------+------------+----+
|  105|Divya|Engineering|      AI Engine| 99000|          48|   1|
|  101| Ravi|Engineering|      AI Engine| 95000|          42|   2|
|  109| Neha|Engineering| Security Suite| 91000|          46|   3|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|   4|
|  110|Farah|         HR|     Onboarding| 60000|          36|   1|
|  107|Priya|         HR|  Policy Revamp| 58000|          37|   2|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|   1|
|  106| Amit|  Marketing|   Social Media| 62000|          35|   2|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|   1|
|  104|Anita|      Sales|Client Outreach| 70000|          38|   2|
+-----+-----+-----------+---------------+------+------------+----+



In [24]:
# 2. Create a new global view with only Engineering department employees
spark.sql("""
  CREATE OR REPLACE GLOBAL TEMP VIEW engineering_only AS
  SELECT * FROM global_temp.employees_global
  WHERE Department = 'Engineering'
""")


DataFrame[]

In [25]:
# Check the new view
spark.sql("SELECT * FROM global_temp.engineering_only").show()


+-----+-----+-----------+--------------+------+------------+
|EmpID| Name| Department|       Project|Salary|HoursPerWeek|
+-----+-----+-----------+--------------+------+------------+
|  101| Ravi|Engineering|     AI Engine| 95000|          42|
|  102|Sneha|Engineering| Data Platform| 87000|          45|
|  105|Divya|Engineering|     AI Engine| 99000|          48|
|  109| Neha|Engineering|Security Suite| 91000|          46|
+-----+-----+-----------+--------------+------+------------+



In [26]:
# 3. Create a global SQL view for employees who work less than 38 hours per week
spark.sql("""
  CREATE OR REPLACE GLOBAL TEMP VIEW active_employees AS
  SELECT * FROM global_temp.employees_global
  WHERE HoursPerWeek < 38
""")


DataFrame[]

In [27]:
# Verify the view
spark.sql("SELECT * FROM global_temp.active_employees").show()


+-----+-----+----------+-------------+------+------------+
|EmpID| Name|Department|      Project|Salary|HoursPerWeek|
+-----+-----+----------+-------------+------+------------+
|  106| Amit| Marketing| Social Media| 62000|          35|
|  107|Priya|        HR|Policy Revamp| 58000|          37|
|  110|Farah|        HR|   Onboarding| 60000|          36|
+-----+-----+----------+-------------+------+------------+

